In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bz2
import gc
import chardet
import re

In [2]:
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')

In [3]:
def get_req_data(req_lines):
    return int(448.374641923*req_lines)
def get_req_data_score(req_lines):
    return int(452.488687783*req_lines)

In [4]:
train_file_lines = train_file.readlines(get_req_data(100000))
test_file_lines = test_file.readlines(get_req_data_score(25000))
del train_file, test_file
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]
print(len(train_file_lines) ,len(test_file_lines))

99255 24914


In [5]:
#Seprating scores from reviews
length_of_train = len(train_file_lines)
length_of_test = len(test_file_lines)
print(length_of_train , length_of_test)
train_file_lines_score = []
test_file_lines_score = []
for i in range(0,length_of_train):
    temp = train_file_lines[i][9]
    train_file_lines_score.append(temp)

for i in range(0,length_of_test):
    temp = test_file_lines[i][9]
    test_file_lines_score.append(temp)

99255 24914


In [6]:
#Converting python list to dataframe
df_train_reviews = pd.DataFrame({'reviews': train_file_lines} )
df_test_reviews = pd.DataFrame({'reviews': test_file_lines} )
df_train_score = pd.DataFrame({'score': train_file_lines_score} )
df_test_score = pd.DataFrame({'score': test_file_lines_score} )

#taking less reviews due to limited computational power
df_train_reviews = df_train_reviews[:75000]
df_test_reviews = df_test_reviews[:25000]
df_train_score = df_train_score[:75000]
df_test_score = df_test_score[:25000]

print(df_train_reviews.shape,df_test_reviews.shape,df_train_score.shape,df_test_score.shape)

(75000, 1) (24914, 1) (75000, 1) (24914, 1)


In [9]:
import re
import nltk
nltk.download('stopwords')
#from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')

#Pre-processing reviews data of traning set
corpus_train = []
for i in range(0, len(df_train_reviews)):
    review = re.sub("<.*?>", "", df_train_reviews['reviews'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    del review[0]
    #ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer() 
    #review = [ps.stem(words) for words in review if not words in set(stopwords.words('english'))]
    review = [lemmatizer.lemmatize(words) for words in review if not words in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_train.append(review)
    
#Pre-processing reviews data of testing set
corpus_test = []
for i in range(0, len(df_test_reviews)):
    review = re.sub("<.*?>", "", df_test_reviews['reviews'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    del review[0]
    #ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer() 
    #review = [ps.stem(words) for words in review if not words in set(stopwords.words('english'))]
    review = [lemmatizer.lemmatize(words) for words in review if not words in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_test.append(review)

corpus_train[0]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shino\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shino\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


'stuning even non gamer sound track beautiful paint senery mind well would recomend even people hate vid game music played game chrono cross game ever played best music back away crude keyboarding take fresher step grate guitar soulful orchestra would impress anyone care listen'

In [10]:
#Bag of words(sparc matrix)
from sklearn.feature_extraction.text import TfidfVectorizer
#cv = TfidfVectorizer(ngram_range=(1,2))
cv = TfidfVectorizer(max_features=5000)
X_train = cv.fit_transform(corpus_train).toarray()
X_test = cv.transform(corpus_test).toarray()
X_train.shape

(75000, 5000)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 0 , max_depth=100)
classifier.fit(X_train, df_train_score)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_test_score, y_pred)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_test_score , y_pred)
accuracy

D:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.8467126916593081